# JijModeling X.XX.X Release Notes

## 機能強化

### エラーメッセージがコード上の位置が含まれるように

`Interpreter.eval_*` 系の関数の投げる例外が、可能な限りソースコード上の位置の情報も出力するようになりました。
ソースコードの詳細な位置情報は Python 3.11 以降でのみ利用可能になるため、より有用な情報を得るには Python 3.11 以降の利用を推奨します。

以下は Python 3.11 でのエラーメッセージの例です：

In [ ]:
import jijmodeling as jm

D = jm.Placeholder("D", dtype=jm.DataType.INTEGER)
T = jm.Placeholder("T", dtype=jm.DataType.INTEGER)
N = jm.Placeholder("N", dtype=jm.DataType.INTEGER)
H = jm.Placeholder("H", shape=(D, T, N))
R = jm.Placeholder("R", shape=(D, N))

x = jm.BinaryVar("x", shape=(D, T, N))
d = jm.Element("d", belong_to=(0, D))
t = jm.Element("t", belong_to=(0, T))
i = jm.Element("i", belong_to=(0, N))

problem = jm.Problem(name="shift_optimization")
problem += jm.Constraint("night_shift", R[d+1, i] <= x[d, T-1, i], forall=[d, i])
problem += jm.sum([d, t, i], x[d, t, i])

problem

In [ ]:
num_days = 7
num_times = 3
num_people = 5

data = problem.generate_random_dataset(
    options={
        "D": {"value": num_days},
        "T": {"value": num_times},
        
        "N": {"value": num_people},
    }
)
interp = jm.Interpreter(data)

try:
    interp.eval_problem(problem)
except Exception as e:
    print(e)

Traceback (most recent last):
    while evaluating Problem `shift_optimization',
        defeind at File "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_49615/2988517098.py", line 14, col 11-48
    while evaluating constraint: `night_shift',
        defeind at File "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_49615/2988517098.py", line 15, col 12-82
    while evaluating expression `R[d + 1, i] - x[d, T - 1, i]',
        defeind at File "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_49615/2988517098.py", line 15, col 41-66
    while evaluating expression `R[d + 1, i]',
        defeind at File "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_49615/2988517098.py", line 15, col 41-50

File "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_49615/2988517098.py", line 15, col 41-50:

    15  |  problem += jm.Constraint("night_shift", R[d+1, i] <= x[d, T-1, i], forall=[d, i])
                                                   ^^^^^^^

Python 3.11未満では、以下のように行単位での情報しか出力されません：

## その他の変更

- JijModeling は ABI3 wheel を提供するようになり、（フリースレッド版でない）Python 3.13もサポートするようになりました！